# Estimating Auto Ownership

This notebook illustrates how to re-estimate a single model component for ActivitySim.  This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [1]:
import os
import larch  # !conda install larch -c conda-forge # for estimation
import pandas as pd

JAX not found. Some functionality will be unavailable.


We'll work in our `test` directory, where ActivitySim has saved the estimation data bundles.

In [2]:
os.chdir('test')

# Load data and prep model for estimation

In [3]:
modelname = "auto_ownership"

from activitysim.estimation.larch import component_model
model, data = component_model(modelname, return_data=True)

loading from output/estimation_data_bundle/auto_ownership/auto_ownership_coefficients.csv
loading from output/estimation_data_bundle/auto_ownership/auto_ownership_coefficients_template.csv
loading from output/estimation_data_bundle/auto_ownership/auto_ownership_values_combined.parquet


# Review data loaded from the EDB

The next step is to read the EDB, including the coefficients, model settings, utilities specification, and chooser and alternative data.

### Coefficients

In [4]:
data.coefficients

,value,constrain
coefficient_name,,
coef_cars1_drivers_2,0.0000,T
coef_cars1_drivers_3,0.0000,T
coef_cars1_persons_16_17,0.0000,T
coef_cars234_asc_marin,0.0000,T
coef_cars1_persons_25_34,0.0000,T
...,...,...
coef_cars4_drivers_3,5.2080,F
coef_cars3_drivers_3,5.5131,F
coef_cars2_drivers_4_up,6.3662,F


#### Utility specification

In [5]:
data.spec

,Label,Description,Expression,cars0,cars1,cars2,cars3,cars4
0,util_drivers_2,2 Adults (age 16+),num_drivers==2,NaN,coef_cars1_drivers_2,coef_cars2_drivers_2,coef_cars3_drivers_2,coef_cars4_drivers_2
1,util_drivers_3,3 Adults (age 16+),num_drivers==3,NaN,coef_cars1_drivers_3,coef_cars2_drivers_3,coef_cars3_drivers_3,coef_cars4_drivers_3
2,util_drivers_4_up,4+ Adults (age 16+),num_drivers>3,NaN,coef_cars1_drivers_4_up,coef_cars2_drivers_4_up,coef_cars3_drivers_4_up,coef_cars4_drivers_4_up
3,util_persons_16_17,Persons age 16-17,num_children_16_to_17,NaN,coef_cars1_persons_16_17,coef_cars2_persons_16_17,coef_cars34_persons_16_17,coef_cars34_persons_16_17
4,util_persons_18_24,Persons age 18-24,num_college_age,NaN,coef_cars1_persons_18_24,coef_cars2_persons_18_24,coef_cars34_persons_18_24,coef_cars34_persons_18_24
5,util_persons_25_34,Persons age 35-34,num_young_adults,NaN,coef_cars1_persons_25_34,coef_cars2_persons_25_34,coef_cars34_persons_25_34,coef_cars34_persons_25_34
6,util_presence_children_0_4,Presence of children age 0-4,num_young_children>0,NaN,coef_cars1_presence_children_0_4,coef_cars234_presence_children_0_4,coef_cars234_presence_children_0_4,coef_cars234_presence_children_0_4
7,util_presence_children_5_17,Presence of children age 5-17,(num_children_5_to_15+num_children_16_to_17)>0,NaN,coef_cars1_presence_children_5_17,coef_cars2_presence_children_5_17,coef_cars34_presence_children_5_17,coef_cars34_presence_children_5_17
8,util_num_workers_clip_3,"Number of workers, capped at 3",@df.num_workers.clip(upper=3),NaN,coef_cars1_num_workers_clip_3,coef_cars2_num_workers_clip_3,coef_cars3_num_workers_clip_3,coef_cars4_num_workers_clip_3
9,util_hh_income_0_30k,"Piecewise Linear household income, $0-30k","@df.income_in_thousands.clip(0, 30)",NaN,coef_cars1_hh_income_0_30k,coef_cars2_hh_income_0_30k,coef_cars3_hh_income_0_30k,coef_cars4_hh_income_0_30k


### Chooser data

In [6]:
data.chooser_data

,model_choice,override_choice,util_drivers_2,util_drivers_3,util_drivers_4_up,util_persons_16_17,util_persons_18_24,util_persons_25_34,util_presence_children_0_4,util_presence_children_5_17,...,auPkTotal,auOpRetail,auOpTotal,trPkRetail,trPkTotal,trOpRetail,trOpTotal,nmRetail,nmTotal,override_choice_code
household_id,,,,,,,,,,,,,,,,,,,,,
5385,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12.181171,9.680380,12.284608,0.000000,0.000000,0.000000,0.000000,1.405790,4.753072,1
6972,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12.160280,9.887764,12.297843,0.783276,2.584280,0.041910,0.304195,3.771265,6.038690,2
9510,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12.362445,10.041750,12.528380,3.956130,6.528868,3.347700,6.315403,5.082520,6.535966,2
11710,1,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,10.858674,9.073761,11.250696,0.000000,0.000000,0.000000,0.000000,2.921278,5.654852,2
12127,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.146401,9.354824,11.606267,0.000000,0.000000,0.000000,0.000000,3.049045,4.203927,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2870016,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.672496,9.621570,12.113200,2.857816,5.487160,2.844021,5.477245,4.156945,6.480754,2
2870309,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.553698,9.538216,11.907127,0.948160,2.979811,0.876269,2.822683,4.672070,7.192998,1
2871512,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.120344,9.325226,11.607641,1.889210,3.947261,1.603548,3.448344,3.564518,6.902982,1


# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [7]:
model.estimate()

,value,best,initvalue,minimum,maximum,nullvalue,holdfast
param_name,,,,,,,
coef_cars1_asc,0.985641,0.985641,1.1865,-50.00,50.00,0.0,0
coef_cars1_asc_county,-0.899184,-0.899184,-0.5660,-50.00,50.00,0.0,0
coef_cars1_asc_marin,-0.605375,-0.605375,-0.2434,-50.00,50.00,0.0,0
coef_cars1_asc_san_francisco,0.189605,0.189605,0.4259,-50.00,50.00,0.0,0
coef_cars1_auto_time_saving_per_worker,0.492575,0.492575,0.4707,-50.00,50.00,0.0,0
...,...,...,...,...,...,...,...
coef_retail_auto_no_workers,0.110968,0.110968,0.0626,-50.00,50.00,0.0,0
coef_retail_auto_workers,0.150951,0.150951,0.1646,-50.00,50.00,0.0,0
coef_retail_non_motor,-0.030000,-0.030000,-0.0300,-0.03,-0.03,0.0,1


┣                x: coef_cars1_asc                            0.985641
┃                   coef_cars1_asc_county                    -0.899184
┃                   coef_cars1_asc_marin                     -0.605375
┃                   coef_cars1_asc_san_francisco              0.189605
┃                   coef_cars1_auto_time_saving_per_worker    0.492575
┃                                                               ...   
┃                   coef_retail_auto_no_workers               0.110968
┃                   coef_retail_auto_workers                  0.150951
┃                   coef_retail_non_motor                    -0.030000
┃                   coef_retail_transit_no_workers           -0.345408
┃                   coef_retail_transit_workers              -0.422654
┃                   Length: 66, dtype: float64
┣          logloss: 0.8965180607424277
┣        d_logloss: coef_cars1_asc                           -0.000081
┃                   coef_cars1_asc_county                     0.000015
┃                   coef_cars1_asc_marin                     -0.000089
┃                   coef_cars1_asc_san_francisco              0.000073
┃                   coef_cars1_auto_time_saving_per_worker    0.000129
┃                                                               ...   
┃                   coef_retail_auto_no_workers               0.000161
┃                   coef_retail_auto_workers                 -0.000123
┃                   coef_retail_non_motor                     0.000000
┃                   coef_retail_transit_no_workers           -0.000328
┃                   coef_retail_transit_workers               0.000328
┃                   Length: 66, dtype: float64
┣              nit: 80
┣             nfev: 96
┣             njev: 80
┣           status: 0
┣          message: 'Optimization terminated successfully'
┣          success: True
┣     elapsed_time: datetime.timedelta(microseconds=267225)
┣           method: 'slsqp'
┣          n_cases: 2000
┣ iteration_number: 80
┣          loglike: -1793.0361214848554

### Estimated coefficients

In [8]:
model.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value,Constrained
Parameter,,,,,,
coef_cars1_asc,0.986,2.71,0.36,,0.00,
coef_cars1_asc_county,-0.899,0.437,-2.06,*,0.00,
coef_cars1_asc_marin,-0.605,0.319,-1.90,,0.00,
coef_cars1_asc_san_francisco,0.190,0.307,0.62,,0.00,
coef_cars1_auto_time_saving_per_worker,0.493,0.598,0.82,,0.00,
coef_cars1_density_0_10_no_workers,0.00,0.00,NA,,0.00,fixed value
coef_cars1_density_10_up_no_workers,-0.0124,0.0116,-1.06,,0.00,
coef_cars1_density_10_up_workers,-0.0265,0.0120,-2.21,*,0.00,
coef_cars1_drivers_2,0.00,0.00,NA,,0.00,fixed value


# Output Estimation Results

In [9]:
from activitysim.estimation.larch import update_coefficients
result_dir = data.edb_directory/"estimated"
update_coefficients(
    model, data, result_dir,
    output_file=f"{modelname}_coefficients_revised.csv",
);

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [10]:
model.to_xlsx(
    result_dir/f"{modelname}_model_estimation.xlsx", 
    data_statistics=False,
)

# Next Steps

The final step is to either manually or automatically copy the `*_coefficients_revised.csv` file to the configs folder, rename it to `*_coefficients.csv`, and run ActivitySim in simulation mode.

In [11]:
pd.read_csv(result_dir/f"{modelname}_coefficients_revised.csv")

,coefficient_name,value,constrain
0,coef_cars1_drivers_2,0.000000,T
1,coef_cars1_drivers_3,0.000000,T
2,coef_cars1_persons_16_17,0.000000,T
3,coef_cars234_asc_marin,0.000000,T
4,coef_cars1_persons_25_34,0.000000,T
...,...,...,...
62,coef_cars4_drivers_3,5.390530,F
63,coef_cars3_drivers_3,6.152139,F
64,coef_cars2_drivers_4_up,6.716321,F
65,coef_cars3_drivers_4_up,9.137024,F


# Modify Spec

Here, we will demonstrate the process of re-estimating the model with a modified
SPEC file.  This does *not* require re-running ActivitySim, it just requires
changing the SPEC file and re-running the Larch estimation only.

Before we start, we will make a copy of the spec file in the EDB directory.
This is not strictly necessary, but since we're about to modify it and
we may want undo our changes, it can be handy to keep a copy of the
original spec file around.  We make the copy here from Python, but a user
can easily make file copies anyhow they wish.

In [12]:
import shutil
if not (data.edb_directory / "auto_ownership_SPEC-ORIGINAL.csv").exists():
    shutil.copy(
        data.edb_directory / "auto_ownership_SPEC.csv", 
        data.edb_directory / "auto_ownership_SPEC-ORIGINAL.csv"
    )

Now, we'll make some changes in the SPEC file.  As an example here, we're going
to re-write the household income section of the file, to change the piecewise 
linear utility from 3 segments to 4.  We'll move the breakpoints and rename some
coefficients to accomodate the change.  As above, for this demo we are editing 
the SPEC file using Python code to make the changes, but a user does not need
to change the file using Python; any CSV editor (e.g. Excel) can be used. 

In [13]:
with open(data.edb_directory / "auto_ownership_SPEC.csv") as f:
    raw_spec = f.read()

orig_lines = """util_hh_income_0_30k,"Piecewise Linear household income, $0-30k","@df.income_in_thousands.clip(0, 30)",,coef_cars1_hh_income_0_30k,coef_cars2_hh_income_0_30k,coef_cars3_hh_income_0_30k,coef_cars4_hh_income_0_30k
util_hh_income_30_75k,"Piecewise Linear household income, $30-75k","@(df.income_in_thousands-30).clip(0, 45)",,coef_cars1_hh_income_30_up,coef_cars2_hh_income_30_up,coef_cars3_hh_income_30_up,coef_cars4_hh_income_30_up
util_hh_income_75k_up,"Piecewise Linear household income, $75k+, capped at $125k","@(df.income_in_thousands-75).clip(0, 50)",,coef_cars1_hh_income_30_up,coef_cars2_hh_income_30_up,coef_cars3_hh_income_30_up,coef_cars4_hh_income_30_up"""

repl_lines = """util_hh_income_0_25k,"Piecewise Linear household income, $0-25k","@df.income_in_thousands.clip(0, 25)",,coef_cars1_hh_income_0_25k,coef_cars2_hh_income_0_25k,coef_cars3_hh_income_0_25k,coef_cars4_hh_income_0_25k
util_hh_income_25_50k,"Piecewise Linear household income, $25-50k","@(df.income_in_thousands-25).clip(0, 25)",,coef_cars1_hh_income_25_50,coef_cars2_hh_income_25_50,coef_cars3_hh_income_25_50,coef_cars4_hh_income_25_50
util_hh_income_50_75k,"Piecewise Linear household income, $50-75k","@(df.income_in_thousands-50).clip(0, 25)",,coef_cars1_hh_income_50_75,coef_cars2_hh_income_50_75,coef_cars3_hh_income_50_75,coef_cars4_hh_income_50_75
util_hh_income_75k_up,"Piecewise Linear household income, $75k+, capped at $150k","@(df.income_in_thousands-75).clip(0, 75)",,coef_cars1_hh_income_75_up,coef_cars2_hh_income_75_up,coef_cars3_hh_income_75_up,coef_cars4_hh_income_75_up"""

raw_spec = raw_spec.replace(orig_lines, repl_lines)

with open(data.edb_directory / "auto_ownership_SPEC.csv", mode="w") as f:
    f.write(raw_spec)


Now to re-estimate the model, we just re-run the same steps as the original estimation above.

In [14]:
model2, data2 = component_model(modelname, return_data=True)

loading from output/estimation_data_bundle/auto_ownership/auto_ownership_coefficients.csv
loading from output/estimation_data_bundle/auto_ownership/auto_ownership_coefficients_template.csv
loading from output/estimation_data_bundle/auto_ownership/auto_ownership_values_combined.parquet


In [15]:
model2.utility_co

DictOfLinearFunction({1: <Empty LinearFunction>, 2:   P.coef_cars1_drivers_2 * X.util_drivers_2
+ P.coef_cars1_drivers_3 * X.util_drivers_3
+ P.coef_cars1_drivers_4_up * X.util_drivers_4_up
+ P.coef_cars1_persons_16_17 * X.util_persons_16_17
+ P.coef_cars1_persons_18_24 * X.util_persons_18_24
+ P.coef_cars1_persons_25_34 * X.util_persons_25_34
+ P.coef_cars1_presence_children_0_4 * X.util_presence_children_0_4
+ P.coef_cars1_presence_children_5_17 * X.util_presence_children_5_17
+ P.coef_cars1_num_workers_clip_3 * X.util_num_workers_clip_3
+ P.coef_cars1_hh_income_0_25k * X('df.income_in_thousands.clip(0, 25)')
+ P.coef_cars1_hh_income_25_50 * X('(df.income_in_thousands-25).clip(0, 25)')
+ P.coef_cars1_hh_income_50_75 * X('(df.income_in_thousands-50).clip(0, 25)')
+ P.coef_cars1_hh_income_75_up * X.util_hh_income_75k_up
+ P.coef_cars1_density_0_10_no_workers * X.util_density_0_10_no_workers
+ P.coef_cars1_density_10_up_no_workers * X.util_density_10_up_no_workers
+ P.coef_cars1_density_0_10_no_workers * X.util_density_0_10_workers
+ P.coef_cars1_density_10_up_workers * X.util_density_10_up_workers
+ P.coef_cars1_asc * X.util_asc
+ P.coef_cars1_asc_san_francisco * X.util_asc_san_francisco
+ P.coef_cars1_asc_county * X.util_asc_solano
+ P.coef_cars1_asc_county * X.util_asc_napa
+ P.coef_cars1_asc_county * X.util_asc_sonoma
+ P.coef_cars1_asc_marin * X.util_asc_marin
+ P.coef_retail_auto_no_workers * X.util_retail_auto_no_workers
+ P.coef_retail_auto_workers * X.util_retail_auto_workers
+ P.coef_retail_transit_no_workers * X.util_retail_transit_no_workers
+ P.coef_retail_transit_workers * X.util_retail_transit_workers
+ P.coef_retail_non_motor * X.util_retail_non_motor_no_workers
+ P.coef_retail_non_motor * X.util_retail_non_motor_workers
+ P.coef_cars1_auto_time_saving_per_worker * X.util_auto_time_saving_per_worker, 3:   P.coef_cars2_drivers_2 * X.util_drivers_2
+ P.coef_cars2_drivers_3 * X.util_drivers_3
+ P.coef_cars2_drivers_4_up * X.util_drivers_4_up
+ P.coef_cars2_persons_16_17 * X.util_persons_16_17
+ P.coef_cars2_persons_18_24 * X.util_persons_18_24
+ P.coef_cars2_persons_25_34 * X.util_persons_25_34
+ P.coef_cars234_presence_children_0_4 * X.util_presence_children_0_4
+ P.coef_cars2_presence_children_5_17 * X.util_presence_children_5_17
+ P.coef_cars2_num_workers_clip_3 * X.util_num_workers_clip_3
+ P.coef_cars2_hh_income_0_25k * X('df.income_in_thousands.clip(0, 25)')
+ P.coef_cars2_hh_income_25_50 * X('(df.income_in_thousands-25).clip(0, 25)')
+ P.coef_cars2_hh_income_50_75 * X('(df.income_in_thousands-50).clip(0, 25)')
+ P.coef_cars2_hh_income_75_up * X.util_hh_income_75k_up
+ P.coef_cars2_density_0_10_no_workers * X.util_density_0_10_no_workers
+ P.coef_cars2_density_10_up_no_workers * X.util_density_10_up_no_workers
+ P.coef_cars2_density_0_10_no_workers * X.util_density_0_10_workers
+ P.coef_cars2_density_10_up_no_workers * X.util_density_10_up_workers
+ P.coef_cars2_asc * X.util_asc
+ P.coef_cars2_asc_san_francisco * X.util_asc_san_francisco
+ P.coef_cars2_asc_county * X.util_asc_solano
+ P.coef_cars2_asc_county * X.util_asc_napa
+ P.coef_cars2_asc_county * X.util_asc_sonoma
+ P.coef_cars234_asc_marin * X.util_asc_marin
+ P.coef_retail_auto_no_workers * X.util_retail_auto_no_workers
+ P.coef_retail_auto_workers * X.util_retail_auto_workers
+ P.coef_retail_transit_no_workers * X.util_retail_transit_no_workers
+ P.coef_retail_transit_workers * X.util_retail_transit_workers
+ P.coef_retail_non_motor * X.util_retail_non_motor_no_workers
+ P.coef_retail_non_motor * X.util_retail_non_motor_workers
+ P.coef_cars2_auto_time_saving_per_worker * X.util_auto_time_saving_per_worker, 4:   P.coef_cars3_drivers_2 * X.util_drivers_2
+ P.coef_cars3_drivers_3 * X.util_drivers_3
+ P.coef_cars3_drivers_4_up * X.util_drivers_4_up
+ P.coef_cars34_persons_16_17 * X.util_persons_16_17
+ P.coef_cars34_persons_18_24 * X.util_persons_18_24
+ P.coef_cars34_persons_25_34 * X.util_persons_25_34
+ P.coef_cars234_presence_children_0_4 * X.util_

In [16]:
model2.estimate()

,value,best,initvalue,minimum,maximum,nullvalue,holdfast
param_name,,,,,,,
coef_cars1_asc,1.088698,1.088698,1.1865,-50.00,50.00,0.0,0
coef_cars1_asc_county,-0.904235,-0.904235,-0.5660,-50.00,50.00,0.0,0
coef_cars1_asc_marin,-0.610738,-0.610738,-0.2434,-50.00,50.00,0.0,0
coef_cars1_asc_san_francisco,0.212794,0.212794,0.4259,-50.00,50.00,0.0,0
coef_cars1_auto_time_saving_per_worker,0.443890,0.443890,0.4707,-50.00,50.00,0.0,0
...,...,...,...,...,...,...,...
coef_retail_auto_no_workers,0.098050,0.098050,0.0626,-50.00,50.00,0.0,0
coef_retail_auto_workers,0.140546,0.140546,0.1646,-50.00,50.00,0.0,0
coef_retail_non_motor,-0.030000,-0.030000,-0.0300,-0.03,-0.03,0.0,1


┣                x: coef_cars1_asc                            1.088698
┃                   coef_cars1_asc_county                    -0.904235
┃                   coef_cars1_asc_marin                     -0.610738
┃                   coef_cars1_asc_san_francisco              0.212794
┃                   coef_cars1_auto_time_saving_per_worker    0.443890
┃                                                               ...   
┃                   coef_retail_auto_no_workers               0.098050
┃                   coef_retail_auto_workers                  0.140546
┃                   coef_retail_non_motor                    -0.030000
┃                   coef_retail_transit_no_workers           -0.341528
┃                   coef_retail_transit_workers              -0.418671
┃                   Length: 74, dtype: float64
┣          logloss: 0.8952428401267184
┣        d_logloss: coef_cars1_asc                           -0.000125
┃                   coef_cars1_asc_county                     0.000036
┃                   coef_cars1_asc_marin                     -0.000055
┃                   coef_cars1_asc_san_francisco             -0.000029
┃                   coef_cars1_auto_time_saving_per_worker    0.000242
┃                                                               ...   
┃                   coef_retail_auto_no_workers               0.000148
┃                   coef_retail_auto_workers                  0.000031
┃                   coef_retail_non_motor                     0.000000
┃                   coef_retail_transit_no_workers           -0.000634
┃                   coef_retail_transit_workers               0.000035
┃                   Length: 74, dtype: float64
┣              nit: 91
┣             nfev: 113
┣             njev: 91
┣           status: 0
┣          message: 'Optimization terminated successfully'
┣          success: True
┣     elapsed_time: datetime.timedelta(microseconds=373152)
┣           method: 'slsqp'
┣          n_cases: 2000
┣ iteration_number: 91
┣          loglike: -1790.4856802534368

We can easily review the parameter estimates from the original and
revised models side by side to see what changed.

In [17]:
with pd.option_context('display.max_rows', 999):
    display(pd.concat({
        "model": model.parameter_summary().data,
        "model2": model2.parameter_summary().data,
    }, axis=1).fillna(""))

model                           \
                                            Value   Std Err  t Stat Signif   
Parameter                                                                    
coef_cars1_asc                              0.986      2.71    0.36          
coef_cars1_asc_county                      -0.899     0.437   -2.06      *   
coef_cars1_asc_marin                       -0.605     0.319   -1.90          
coef_cars1_asc_san_francisco                0.190     0.307    0.62          
coef_cars1_auto_time_saving_per_worker      0.493     0.598    0.82          
coef_cars1_density_0_10_no_workers           0.00      0.00      NA          
coef_cars1_density_10_up_no_workers       -0.0124    0.0116   -1.06          
coef_cars1_density_10_up_workers          -0.0265    0.0120   -2.21      *   
coef_cars1_drivers_2                         0.00      0.00      NA          
coef_cars1_drivers_3                         0.00      0.00      NA          
coef_cars1_drivers_4_up                      2.57      1.16    2.23      *   
coef_cars1_hh_income_0_30k                 0.0401   0.00923    4.34    ***   
coef_cars1_hh_income_30_up                   0.00      0.00      NA          
coef_cars1_num_workers_clip_3                0.00      0.00      NA          
coef_cars1_persons_16_17                     0.00      0.00      NA          
coef_cars1_persons_18_24                   -0.522     0.196   -2.67     **   
coef_cars1_persons_25_34                     0.00      0.00      NA          
coef_cars1_presence_children_0_4           0.0585     0.409    0.14          
coef_cars1_presence_children_5_17          0.0358     0.292    0.12          
coef_cars234_asc_marin                       0.00      0.00      NA          
coef_cars234_presence_children_0_4          0.714     0.438    1.63          
coef_cars2_asc                              -1.43      2.71   -0.53          
coef_cars2_asc_county                      -0.594     0.468   -1.27          
coef_cars2_asc_san_francisco                0.739     0.387    1.91          
coef_cars2_auto_time_saving_per_worker      0.949     0.626    1.52          
coef_cars2_density_0_10_no_workers         -0.164    0.0382   -4.29    ***   
coef_cars2_density_10_up_no_workers        -0.218    0.0538   -4.05    ***   
coef_cars2_drivers_2                         3.12     0.192   16.24    ***   
coef_cars2_drivers_3                         3.80     0.339   11.21    ***   
coef_cars2_drivers_4_up                      6.72      1.21    5.57    ***   
coef_cars2_hh_income_0_30k                 0.0446    0.0133    3.35    ***   
coef_cars2_hh_income_30_up                0.00601   0.00258    2.33      *   
coef_cars2_num_workers_clip_3               0.468     0.141    3.31    ***   
coef_cars2_persons_16_17                   -0.892     0.334   -2.67     **   
coef_cars2_persons_18_24                    -1.27     0.225   -5.63    ***   
coef_cars2_persons_25_34                   -0.450     0.113   -4.00    ***   
coef_cars2_presence_children_5_17           0.115     0.331    0.35          
coef_cars34_asc_county                  -0.000199     0.493   -0.00          
coef_cars34_asc_san_francisco              0.0344     0.504    0.07          
coef_cars34_density_0_10_no_workers        -0.325    0.0607   -5.35    ***   
coef_cars34_density_10_up_no_workers       -0.511     0.310   -1.65          
coef_cars34_persons_16_17                   -1.88     0.385   -4.88    ***   
coef_cars34_persons_18_24                   -1.17     0.235   -4.98    ***   
coef_cars34_persons_25_34                  -0.910     0.142   -6.41    ***   
coef_cars34_presence_children_5_17          0.158     0.358    0.44          
coef_cars3_asc                              -3.31      2.76   -1.20          
coef_cars3_auto_time_saving_per_worker      0.487     0.680    0.72          
coef_cars3_drivers_2                         3.46     0.442    7.83    ***   
coef_cars3_drivers_3                         6.15     0.525   11.7

In [18]:
with pd.option_context('display.max_rows', 999):
    display(pd.concat({
        "model": model.estimation_statistics_raw(),
        "model2": model2.estimation_statistics_raw(),
    }, axis=1).fillna(""))

model       model2
Number of Cases                    Aggregate  2000.000000  2000.000000
Log Likelihood at Convergence      Aggregate -1793.036121 -1790.485680
                                   Per Case     -0.896518    -0.895243
Log Likelihood at Null Parameters  Aggregate -3245.057820 -3245.057820
                                   Per Case     -1.622529    -1.622529
Rho Squared w.r.t. Null Parameters Aggregate     0.447456     0.448242